# Evaluates the performance of Logistic Regression Classifier with different training sizes of diferrent datasets

In [72]:
import pandas as pd


In [73]:
# GTZAN_dataset = pd.read_csv("../data/gtzan/features_30_sec.csv")
# GTZAN_dataset.drop(['filename','length'], axis =1, inplace = True)
# X = GTZAN_dataset.drop(['label'], axis=1)
# y = GTZAN_dataset['label']

In [74]:
fma = pd.read_csv('../data/fma/fmaManipulated.csv')
fma_small = fma[fma['subset'] == 'large']
fma_small.reset_index(inplace=True, drop=True)
fma_small.head()

,track_id,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6,subset,genre_top
0,135,0.978402,0.623572,1.129261,-1.043970,-1.127288,-0.032642,-0.878434,-0.916234,-0.743257,...,0.020464,-0.038451,0.234863,0.050837,0.050781,0.003418,0.297873,0.024899,large,Rock
1,137,1.607191,1.199649,0.745765,1.430835,-0.155455,0.768487,3.005594,2.779431,1.567088,...,0.011149,40.950294,0.619629,0.088439,0.086914,0.000000,3.275552,0.021831,large,Experimental
2,138,1.501403,0.957514,1.276685,0.640389,0.442936,0.845533,0.917440,0.770691,1.787260,...,0.012725,5.126394,0.627441,0.102232,0.099121,0.004883,0.633483,0.034676,large,Experimental
3,142,-0.581270,3.199484,1.298346,-0.681253,-0.935093,-0.960304,-0.748014,-0.715798,-0.905853,...,0.022975,4.865969,0.309570,0.066520,0.061035,0.001953,1.625022,0.036644,large,Folk
4,144,0.121207,-0.025308,0.541327,0.967687,1.082456,1.675841,1.741263,0.470098,4.051888,...,0.018568,0.627553,0.239258,0.079759,0.078613,0.000000,0.226211,0.034378,large,Jazz


In [75]:
X = fma_small.drop(['track_id','genre_top','subset'], axis=1)
y = fma_small['genre_top']

In [76]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import json

Arguments:

(X) Matrix of features

(y) Array of targets

(train_size) Size of the trainset, this trian set will be spllited into more partitions  

In [77]:
def performanceOfTrainSizes(X, y, train_size):
    pipeline = Pipeline([
                            ('StandardScaler',  StandardScaler()),
                            ('meu_classificador', LogisticRegression(max_iter=10000))
                        ])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, stratify=y)

    n_train = np.logspace(-2, np.log10(0.8), 10)
    n_train_ = [int(n*X_train.shape[0]) for n in n_train]

    n_train_ = [n for n in n_train_ if n > len(set(y))]
    acc_mean = []
    acc_std = []
    acc_mean_overfit = []
    acc_std_overfit = []
    
    for n in n_train_:
        this_acc = []
        this_acc_overfit = []
        for k in tqdm(range(50)):
            X_train_, _, y_train_, _ = train_test_split(X_train, y_train, train_size=n, stratify=y_train)
            pipeline.fit(X_train_,y_train_)
            
            y_pred = pipeline.predict(X_test)
            
            acc = accuracy_score(y_pred,y_test)

            y_pred_overfit = pipeline.predict(X_train)
            acc_overfit = accuracy_score(y_pred_overfit,y_train)

            this_acc_overfit.append(acc_overfit)
            this_acc.append(acc)
        
        this_acc_overfit = np.array(this_acc_overfit)
        this_acc = np.array(this_acc)
        acc_mean.append(this_acc.mean())
        acc_std.append(this_acc.std())
        acc_mean_overfit.append(this_acc_overfit.mean())
        acc_std_overfit.append(this_acc_overfit.std())
    return acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train_

In [78]:
def saveMetricsStorage(storageJsonPath ,datasetName, acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train_,datasetSize):
    with open(storageJsonPath, 'r') as f:
        storage = json.load(f)
    
    storage[datasetName] = {
        'acc_mean': acc_mean,
        'acc_mean_overfit': acc_mean_overfit,
        'acc_std': acc_std,
        'acc_std_overfit': acc_std_overfit,
        'n_train_': n_train_,
        'datasetSize': datasetSize
    }
    
    with open(storageJsonPath, 'w') as f:
        json.dump(storage, f)

In [79]:
acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train_ = performanceOfTrainSizes(X, y, 0.8)
saveMetricsStorage('../data/metricsStorage.json', 'FMA-large', acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train_,len(y))

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [38:22<00:00, 46.06s/it]


In [80]:
acc_mean

[0.49486196319018405,
 0.509258691206544,
 0.524258691206544,
 0.529780163599182,
 0.5378527607361963,
 0.5410173824130879,
 0.5614723926380368,
 0.594739263803681,
 0.6258026584867076,
 0.6501891615541924]

In [81]:
acc_mean_overfit 

[0.49883691206543973,
 0.515329754601227,
 0.534924591002045,
 0.5482016871165645,
 0.567321063394683,
 0.5892714723926381,
 0.6232387525562373,
 0.6656978527607363,
 0.7035058793456033,
 0.7329703476482617]

In [82]:
acc_std

[0.012953263916789488,
 0.012899178269244372,
 0.010536938805297618,
 0.01047124753490078,
 0.010299046445425003,
 0.009391548748333384,
 0.006894876438113353,
 0.005589197334477628,
 0.0046284160332959404,
 0.003065016901015458]

In [83]:
acc_std_overfit

[0.012688306442401789,
 0.012364498816831304,
 0.00999332857326273,
 0.007934376780819579,
 0.007374192350145336,
 0.0063467802841144895,
 0.0037676577995119707,
 0.0032745628254856403,
 0.002573186713423827,
 0.0015693422947931559]

In [84]:
n_train_

[156, 254, 414, 674, 1097, 1785, 2905, 4727, 7692, 12518]